In [1]:
import pickle
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, roc_auc_score, confusion_matrix

from sklearn.model_selection import GridSearchCV

In [2]:
# files in output to use
target_in_output = ("outputs/XGBoost")
#seed data folder
seed_data=("data/ml_paper_seeds_all.csv")

In [17]:
species=["Alnus_glutinosa","Betula_pendula","Betula_pubescens","Pinus_sylvestris","Sorbus_aucuparia","all_species"]
folders=["names_all_features","names_colour_features","names_xray_features"]

feature_sets = {
    "names_all_features": ["Area", "Perim.", "Feret", "MinFeret", "Circ.", "AR", "Round", "Solidity", "Mean_L", "Mean_a", "Mean_b", "Mean_grey", "Mean_core_grey", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM", "Bin_germ"],
    "names_colour_features": ["Area", "Perim.", "Feret", "MinFeret", "Circ.", "AR", "Round", "Solidity", "Mean_L", "Mean_a", "Mean_b", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM", "Bin_germ"],
    "names_xray_features": ["Mean_grey", "Mean_core_grey", "Bin_germ"]
}

In [18]:
results_list = []


# Filter out 'Bin_germ' from the list of features for feature importance
filtered_features = ["Area", "Perim.", "Feret", "MinFeret", "Circ.", "AR", "Round", "Solidity", "Mean_L", "Mean_a", "Mean_b", "Mean_grey", "Mean_core_grey", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM"]


for folder in folders:
    all_yhat = []
    for specie in species:
        # Combine species and features here
        combined_path = os.path.join(target_in_output,specie, folder, "cv_model.dat")
        
        # Load data from data.dat
        with open(combined_path, 'rb') as file:
            loaded_model = pickle.load(file)
                
        # Use the loaded data as needed, for example, print it
        #print(f"Data for {specie} in {folder}: {loaded_model}")

        features = feature_sets.get(folder, [])
            
        seeds_all_data = pd.read_csv(seed_data)
        seeds_sp = seeds_all_data[seeds_all_data.Species == specie] 
        seeds = seeds_sp[seeds_sp.Set == "train"] 
        seeds = seeds[features]
        X, y = seeds[seeds.columns.tolist()[:-1]], seeds[seeds.columns.tolist()[-1]]
        X = X.to_numpy() #RDCOMM if you leave it as a Pandas df, the model still runs and the features keep their names for feature importances (win win!)
        y = y.to_numpy()
        holdout = seeds_sp[seeds_sp.Set == "Hold out"] 
        holdout = holdout[features]
        X_test, y_test = holdout[holdout.columns.tolist()[:-1]], holdout[holdout.columns.tolist()[-1]]
        X_test = X_test.to_numpy()
        y_test = y_test.to_numpy()

        yhat = loaded_model.predict(X_test)
        yhat_probs = loaded_model.predict_proba(X_test)[:, 1] # Probability of positive class to calculate AUC, but I think in this case it is the same as yhat. Probably the threshold has already been applied.
        #evaluate the model
        acc = accuracy_score(y_test, yhat)
        prec = precision_score(y_test, yhat)
        f1 = f1_score(y_test, yhat)
        rec = recall_score(y_test, yhat) #same as sensitivity
        auc = roc_auc_score(y_test, yhat_probs) #AUC improves considerably when XGBClassifier(objective = 'binary:logistic')

        # Calculate specificity
        tn, fp, fn, tp = confusion_matrix(y_test, yhat).ravel()
        specif = tn / (tn + fp)

        cm = confusion_matrix(y_test, yhat)

        ### RDCOMM: changed from here
        # results_list.append({'Species':specie, 'model':folder, 'Accuracy': acc, 'Precision': prec, 'f1': f1,'Recall':rec,'AUC':auc,'Specificity':specif,'cm_tl':cm[0][0],
        #                      'cm_tr':cm[0][1],'cm_bl':cm[1][0],'cm_br':cm[1][1]})

        fi_gain = loaded_model.best_estimator_.get_booster().get_score(importance_type='gain')
        fi_weight = loaded_model.best_estimator_.get_booster().get_score(importance_type='weight')

        
        # Would like to append the score from fi_gain and fi_weights to results_list. Note that not all models will have all features.
        # Append results to the list
        results_dict = {
            'Species': specie,
            'model': folder,
            'Accuracy': acc,
            'Precision': prec,
            'f1': f1,
            'Recall': rec,
            'AUC': auc,
            'Specificity': specif,
            'cm_TN_tl': cm[0][0],
            'cm_FP_tr': cm[0][1],
            'cm_FN_bl': cm[1][0],
            'cm_TP_br': cm[1][1],
        }

        # Add feature importance scores to the results_dict (excluding 'Bin_germ')
        results_dict.update({'fi_gain_' + str(feature): fi_gain.get(feature, 'NA') for feature in filtered_features})
        results_dict.update({'fi_weight_' + str(feature): fi_weight.get(feature, 'NA') for feature in filtered_features})

        results_list.append(results_dict)

        print('Accuracy: %.3f, Precision: %.3f, f1: %.3f, Recall: %.3f, AUC: %.3f, Specificity: %.3f' % (acc, prec, f1, rec, auc, specif))

        all_yhat.extend(yhat)
    
    results_test = pd.DataFrame({'Prediction': all_yhat})
    output_file = f'Outputs/XGBoost/predictions_{folder}.csv'
    results_test.to_csv(output_file, index=False)
        

final_df = pd.DataFrame(results_list)
csv_filename = ("outputs/XGBoost/evaluation_results.csv")
final_df.to_csv(csv_filename, index=False)          
                    


Accuracy: 0.975, Precision: 0.974, f1: 0.984, Recall: 0.993, AUC: 0.956, Specificity: 0.918
Accuracy: 0.824, Precision: 0.805, f1: 0.873, Recall: 0.954, AUC: 0.777, Specificity: 0.600
Accuracy: 0.739, Precision: 0.718, f1: 0.818, Recall: 0.951, AUC: 0.673, Specificity: 0.395
Accuracy: 0.823, Precision: 0.810, f1: 0.883, Recall: 0.971, AUC: 0.735, Specificity: 0.500
Accuracy: 0.915, Precision: 0.929, f1: 0.956, Recall: 0.984, AUC: 0.492, Specificity: 0.000
Accuracy: 0.855, Precision: 0.860, f1: 0.905, Recall: 0.955, AUC: 0.773, Specificity: 0.591
Accuracy: 0.751, Precision: 0.804, f1: 0.844, Recall: 0.888, AUC: 0.607, Specificity: 0.327
Accuracy: 0.629, Precision: 0.661, f1: 0.745, Recall: 0.854, AUC: 0.547, Specificity: 0.240
Accuracy: 0.668, Precision: 0.673, f1: 0.771, Recall: 0.902, AUC: 0.596, Specificity: 0.289
Accuracy: 0.763, Precision: 0.783, f1: 0.840, Recall: 0.904, AUC: 0.678, Specificity: 0.452
Accuracy: 0.925, Precision: 0.930, f1: 0.961, Recall: 0.995, AUC: 0.497, Specifi